In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn import preprocessing
from sklearn import decomposition
import matplotlib.animation as animation
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn.metrics import silhouette_score
import seaborn as sns
from scipy.stats import kruskal
import graphviz as gr
import math
random.seed(1)

## Reading data from CSV 

In [ ]:
type1_learners = [7.0,8.0,9.0,10.0,12.0,25.0,30.0,31.0,32.0,38.0,40.0,42.0,44.0,47.0]  #Expressive Explorers

In [ ]:
dataset_path = 'path/to/dataset.csv'

In [ ]:
## Reading data
behavioral_data = pd.read_csv('PE-HRI_behavioral_timeseries.csv')
behavioral_data=behavioral_data.drop('Unnamed: 0',axis=1)
behavioral_data=behavioral_data.dropna()
behavioral_data=behavioral_data[behavioral_data['team'].isin(type1_learners)]
behavioral_data

In [ ]:
## Separating the metadata
meta_data=behavioral_data.loc[:,'team':'window']
team_id = behavioral_data.loc[:,'team']

In [ ]:
## Separating features and their columns' names
features = behavioral_data.loc[:,'T_add':'normalized_time']
columns= features.columns

## Standarizing data

In [ ]:
## Normalizing behavioral features wit a MinMax Scaler
def standarize(df):
    standardiser = preprocessing.MinMaxScaler()
    data = standardiser.fit_transform(df)
    df =pd.DataFrame(data,columns= df.columns)
    return df

In [ ]:
## Standarized data
std_features=standarize(features)
std_features

## PCA

In [ ]:
def PCA(df,n_components=columns.shape[0]):
    pca = decomposition.PCA(n_components=n_components)
    data=pca.fit_transform(df)
    df=pd.DataFrame(data)
    return df,pca.explained_variance_ratio_

In [ ]:
PCAfeatures,varRatio= PCA(std_features)

In [ ]:
fig,ax = plt.subplots()
x = np.arange(1,varRatio.shape[0]+1,step=1)
ax.bar(x,varRatio)
plt.xlabel('Principle Components')
plt.ylabel('Proportion')
plt.title('Explained Variance')
plt.show()

based on the elbow method on the proportion of explained variance, we consider the first 4 PCs

In [ ]:
# We take the first 4 components
features_afterPCA,var = PCA(std_features, n_components=4)
features_afterPCA

In [ ]:
## Pair Plot of the PCs
sns.pairplot(features_afterPCA)

## K-Means Clustering

In [ ]:
# K-means clustering with k from [1,10]
# compute inertia and silhouette_scores to choose optimal k
sse = []
s_score=[]
list_k = np.arange(1,10)
for k in list_k:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(features_afterPCA)
    labels = kmeans.labels_
    sse.append(kmeans.inertia_)
    if(k!=1):
        s_score.append(silhouette_score(features_afterPCA,labels))


    

In [ ]:
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance')
plt.title('intertia to number of clusters')

In [ ]:
plt.plot(np.arange(2,10), s_score, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('silhouette score')
plt.title('silhouette to number of clusters')

We can conclude that there are 2 clusters

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(features_afterPCA)
labels = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

In [ ]:
cluster_centers

In [ ]:
## features and their labels
clustered_features=pd.DataFrame(std_features)
clustered_features = clustered_features.assign(label=pd.Series(labels).values)
clustered_features = clustered_features.assign(team=pd.Series(team_id).values)

In [ ]:
## size per cluster
clusters= clustered_features.groupby('label')
size_per_cluster =clusters.size()

In [ ]:
## conditional means
mk = clusters.mean()
mk

#### Cluster Analysis

In [ ]:
## Cluster 0
clusterArray=[]
cluster0 = clusters.get_group(0).drop('label',axis='columns')
clusterArray.append(cluster0)


In [ ]:
## Cluster 1
cluster1 = clusters.get_group(1).drop('label',axis='columns')
clusterArray.append(cluster1)

In [ ]:
## Kruskal Wallis test on the clusters
clus01=[]

for feature in columns:
    clus01.append((feature,kruskal(clusterArray[0][feature],clusterArray[1][feature]).pvalue))



In [ ]:
clus01.sort(key= lambda elem : elem[1] )

In [ ]:
## sorted by ascendant p-value/// features with pvalue > threshold are marked with xx
threshold=0.01
print (' ---- for cluster 0 and cluster 1 :')
for pair in clus01:
    ind=''
    if(pair[1]>threshold):
        ind=' xx '
    print(' ------'+ind+' feature:'+str(pair[0])+'  || pvalue : '+str(pair[1]))
    


In [ ]:
significant_feature01 =list(map(lambda c: c[0],list(filter(lambda pt: pt[1]<threshold,clus01))))
mean0=list()
mean1=list()
for ft in significant_feature01:
    mean0.append(mk[ft][0])
    mean1.append(mk[ft][1])
x = np.arange(len(significant_feature01))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(15,7))
rects1 = ax.bar(x - width/2, mean0, width, label='Cluster0')
rects2 = ax.bar(x + width/2, mean1, width, label='Cluster1')

ax.set_ylabel('Means')
ax.set_title('Means for significant features per cluster')
ax.set_xticks(x)
ax.set_xticklabels(significant_feature01)
plt.xticks(rotation=90)
ax.legend()


plt.show()


## HMM

In [ ]:
from hmmlearn import hmm

In [ ]:
## For each team the set of observations is a sequence
## So we have #teams sequences with #dp_per_team emissions
## label and team_n should be dropped and obseravtions should be sorted by time
by_team = clustered_features.drop('label',axis=1).groupby('team')
lengths= by_team.count()['normalized_time'].to_numpy()
observations = by_team.get_group(7).drop('team',axis=1).to_numpy()
for name,group in by_team:
    if(name!=7):
        observations = np.concatenate([observations,group.drop('team',axis=1).to_numpy()])

In [ ]:
# build HMM model with 3 components and fit it to the observations sequences
model = hmm.GMMHMM(n_components=3,covariance_type='spherical',random_state= random.randint(0,100))
model = model.fit(observations,lengths)

In [ ]:
## Indicates whether the model has converged
print('The model has converged: {}'.format(model.monitor_.converged))

In [ ]:
## Initial state occupation distribution.
model.startprob_

In [ ]:
## Matrix of transition probabilities between states.
model.transmat_

In [ ]:
## predict each dp state
states=[]
for name,group in by_team :
    obs= group.drop('team',axis=1).to_numpy()
    states =  np.concatenate([states,model.predict(obs,lengths=[obs.shape[0]])])
states_d= pd.DataFrame(std_features).assign(state=pd.Series(states).values)
states_dis = states_d.groupby('state')

In [ ]:
state0 = states_dis.get_group(0)
state1 = states_dis.get_group(1)
state2 = states_dis.get_group(2)
states_dis.size()

#### Behaviors analysis

In [ ]:
states= list(range(3))
## Mean parameters for each mixture component in each state.
fig, axs = plt.subplots(math.ceil(columns.shape[0]/4.0),4,figsize=(15,20))
for i in range(len(model.means_[0][0])):
    axs[i//4,i%4].bar(states,model.means_[:,0,i])
    axs[i//4,i%4].set_ylabel('Means')
    axs[i//4,i%4].set_title('Means for feature : '+columns[i])
plt.show()

In [ ]:
pvalues01 = []
pvalues02= []
pvalues12 = []
pvalues012=[]
for feature in columns:
    try:
        pvalues01.append(kruskal(state0[feature],state1[feature]).pvalue)
    except:
        pvalues01.append(None)
    try :
        pvalues02.append(kruskal(state0[feature],state2[feature]).pvalue)
    except :
        pvalues02.append(None)
    try:
        pvalues12.append(kruskal(state1[feature],state2[feature]).pvalue)
    except:
        pvalues12.append(None)
    try:
        pvalues012.append(kruskal(state0[feature],state1[feature],state2[feature]).pvalue)
    except:
        pvalues012.append(None)
tests_df = pd.DataFrame(index=columns).assign(pvalue_01=pd.Series(pvalues01).values).assign(pvalue_02=pd.Series(pvalues02).values)
tests_df = tests_df.assign(pvalue_12=pd.Series(pvalues12).values).assign(pvalue_012=pd.Series(pvalues012).values)
tests_df = tests_df.assign(mean_0=pd.Series(model.means_[0][0]).values).assign(mean_1=pd.Series(model.means_[1][0]).values)
tests_df = tests_df.assign(mean_2=pd.Series(model.means_[2][0]).values)
tests_df=tests_df.sort_values('pvalue_012')
tests_df

In [ ]:
filtered= tests_df[tests_df['pvalue_01']<0.01]
filtered=filtered.sort_values('pvalue_01')
significant = filtered.index.to_list()
mean0=filtered['mean_0'].to_list()
mean1=filtered['mean_1'].to_list()
x = np.arange(len(significant))
width = 0.35  # the width of the bars
fig, ax = plt.subplots(figsize=(20,15))
rects1 = ax.bar(x - width/2, mean0, width, label='State0')
rects1 = ax.bar(x + width/2, mean1, width, label='State1')
ax.set_ylabel('Means')
ax.set_title('Means for significant features per state')
ax.set_xticks(x)
ax.set_xticklabels(significant)
plt.xticks(rotation=90)
ax.legend()


plt.show()

In [ ]:
filtered= tests_df[tests_df['pvalue_02']<0.01]
filtered=filtered.sort_values('pvalue_02')
significant = filtered.index.to_list()
mean0=filtered['mean_0'].to_list()
mean2=filtered['mean_2'].to_list()
x = np.arange(len(significant))
width = 0.35  # the width of the bars
fig, ax = plt.subplots(figsize=(20,15))
rects0 = ax.bar(x - width/2, mean0, width, label='State0')
rects2 = ax.bar(x + width/2, mean2, width, label='State2')
ax.set_ylabel('Means')
ax.set_title('Means for significant features per state')
ax.set_xticks(x)
ax.set_xticklabels(significant)
plt.xticks(rotation=90)
ax.legend()


plt.show()

In [ ]:
filtered= tests_df[tests_df['pvalue_12']<0.01]
filtered=filtered.sort_values('pvalue_12')
significant = filtered.index.to_list()
mean1=filtered['mean_1'].to_list()
mean2=filtered['mean_2'].to_list()
x = np.arange(len(significant))
width = 0.35  # the width of the bars
fig, ax = plt.subplots(figsize=(20,15))
rects1 = ax.bar(x - width/2, mean1, width, label='State1')
rects2 = ax.bar(x + width/2, mean2, width, label='State2')
ax.set_ylabel('Means')
ax.set_title('Means for significant features per state')
ax.set_xticks(x)
ax.set_xticklabels(significant)
plt.xticks(rotation=90)
ax.legend()


plt.show()

In [ ]:
filtered= tests_df[tests_df['pvalue_012']<0.01]
significant = filtered.index.to_list()
mean0=filtered['mean_0'].to_list()
mean1=filtered['mean_1'].to_list()
mean2=filtered['mean_2'].to_list()

x = np.arange(len(significant))
x=x+0.5# the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=(20,15))
rects1 = ax.bar(x - width, mean0, width, label='State0')
rects1 = ax.bar(x , mean1, width, label='State1')
rects2 = ax.bar(x + width, mean2, width, label='State2')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Means')
ax.set_title('Means for significant features per state')
ax.set_xticks(x)
ax.set_xticklabels(significant)
plt.xticks(rotation=90)
ax.legend()


plt.show()

#### Generate state description and transitions graph

In [ ]:
log=['T_add', 'T_remove', 'T_ratio_add_del', 'T_action', 'T_hist', 'T_help',
       'T1_T1_rem', 'T1_T1_add', 'T1_T2_rem', 'T1_T2_add', 'redundant_exist']
video=['positive', 'negative', 'arousal', 'positive_minus_negative', 'smile',
       'screen_left', 'screen_right', 'at_partner', 'at_robot', 'other']
audio=['T_speech', 'T_silence', 'T_overlap', 'T_short_pauses', 'T_long_pauses',
       'T_overlap_over_speech']
sort_by=filtered.index.map(lambda x: 0 if(x in log) else (1 if(x in video) else (2 if(x in audio) else 3)))
graph = filtered.assign(sort_by=sort_by)
graph=graph.sort_values('sort_by',axis=0)
graph=graph.drop('smile',axis=0)
description = ['''Highest''','''High''','''Medium''','''Low''','''Lowest''']
mean0=graph['mean_0'].to_list()
mean1=graph['mean_1'].to_list()
mean2=graph['mean_2'].to_list()
means=[mean0,mean1,mean2]
significant=graph.index.to_list()
colors= ['''''','''''','''''']

In [ ]:
#Generate state description
# Highest Value for each feature is marked with highest, same for lowest
# the remaining value is High, Medium, Low based on its distance from the min and the max
states_des=[["","",""],["","",""],["","",""],["","",""]]
new_des=["","",""]
i=0
for ft in significant:
    max_value= max(means[0][i],means[1][i],means[2][i])
    min_value=min(means[0][i],means[1][i],means[2][i])
    for j in range(3):
        if(means[j][i]==max_value):
            new_des[j]= "{:<30}".format(ft) +'\t'+ description[0] +"<br/>"
        elif(((means[j][i]-min_value)/(max_value-min_value))>2/3):
            new_des[j]= "{:<30}".format(ft) +'\t'+ description[1] +"<br/>"
        elif (((means[j][i]-min_value)/(max_value-min_value))>1/3):
            new_des[j]= "{:<30}".format(ft) +'\t'+ description[2] +"<br/>"
        elif(means[j][i]==min_value):
            new_des[j]= "{:<30}".format(ft) +'\t'+ description[4] +"<br/>"
        else:
            new_des[j]= "{:<30}".format(ft)+'\t'+ description[3] +"<br/>"
    if not(new_des[0]==new_des[1] and new_des[0]==new_des[2] and new_des[2]==new_des[1]):
        for j in range(3):
            states_des[graph.loc[ft,'sort_by']][j]+=new_des[j]
    i+=1

In [ ]:
G= gr.Digraph('Type1HMMStateDiagram',format='jpeg')
G.attr('graph',pad='1',ranksep='1',nodesep='1')
prob="{proba:.2e}"

widths= [4,4,4]
widths[np.argmin(model.startprob_)] = 0.5
widths[np.argmax(model.startprob_)]=6

G.attr('node',color='red')
G.node('0.0','''<<font color="blue">''' + states_des[0][0]+'''</font>'''\
       +'''<font color="#1d8348">'''+ states_des[1][0]+'''</font>'''\
       +'''<font color="orange">'''+ states_des[2][0]+'''</font>'''\
       '''<font color="black">'''+ states_des[3][0]+'''</font>>''',shape='box')

G.node('1.0','''<<font color="blue">''' + states_des[0][1]+'''</font>'''\
       +'''<font color="#1d8348">'''+ states_des[1][1]+'''</font>'''\
       +'''<font color="orange">'''+ states_des[2][1]+'''</font>'''\
       '''<font color="black">'''+ states_des[3][1]+'''</font>>''',shape='box')

G.node('2.0','''<<font color="blue">''' + states_des[0][2]+'''</font>'''\
       +'''<font color="#1d8348">'''+ states_des[1][2]+'''</font>'''\
       +'''<font color="orange">'''+ states_des[2][2]+'''</font>'''\
       '''<font color="black">'''+ states_des[3][2]+'''</font>>''',shape='box')

G.attr('node',penwidth=str(widths[0]))
G.node('0','State 0\n\nInitial state probability: '+prob.format(proba=model.startprob_[0]))
G.attr('node',penwidth=str(widths[1]))
G.node('1','State 1\n\nInitial state probability: '+prob.format(proba=model.startprob_[1]))
G.attr('node',penwidth=str(widths[2]))
G.node('2','State 2\n\nInitial state probability: '+prob.format(proba=model.startprob_[2]))


prob="{proba:.3f}"
G.attr('edge',penwidth=str(10*model.transmat_[0][0]))
G.edge('0','0',prob.format(proba=model.transmat_[0][0]))
G.attr('edge',penwidth=str(10*model.transmat_[0][1]))
G.edge('0','1',prob.format(proba=model.transmat_[0][1]))
G.attr('edge',penwidth=str(10*model.transmat_[0][2]))
G.edge('0','2',prob.format(proba=model.transmat_[0][2]))

G.attr('edge',penwidth=str(10*model.transmat_[1][0]))
G.edge('1','0',prob.format(proba=model.transmat_[1][0]))
G.attr('edge',penwidth=str(10*model.transmat_[1][1]))
G.edge('1','1',prob.format(proba=model.transmat_[1][1]))
G.attr('edge',penwidth=str(10*model.transmat_[1][2]))
G.edge('1','2',prob.format(proba=model.transmat_[1][2]))

G.attr('edge',penwidth=str(10*model.transmat_[2][0]))
G.edge('2','0',prob.format(proba=model.transmat_[2][0]))
G.attr('edge',penwidth=str(10*model.transmat_[2][1]))
G.edge('2','1',prob.format(proba=model.transmat_[2][1]))
G.attr('edge',penwidth=str(10*model.transmat_[2][2]))
G.edge('2','2',prob.format(proba=model.transmat_[2][2]))

G.attr('edge',penwidth=str(1))
G.attr('edge',fontcolor='red')
G.attr('edge',dir='none')
G.attr('edge',color='red')
G.edge('0','0.0','Key Features')
G.edge('1','1.0','Key Features')
G.edge('2','2.0','Key Features')
G.graph_attr.update(size="15,12")

G.view()

